## Introduction

Time series analysis refers to the analysis of change in the trend of the data over a period of time. Time series analysis has a variety of applications. One such application is the prediction of the future value of an item based on its past values. Future stock price prediction is probably the best example of such an application. In this article, we will see how we can perform time series analysis with the help of a recurrent neural network (RNN). We will be predicting the future stock prices of the Apple Company (AAPL), based on its stock prices of the past 5 years.

Reference: [Time Series Analysis with LSTM using Python's Keras Library](https://stackabuse.com/time-series-analysis-with-lstm-using-pythons-keras-library/)

In [1]:
#import libraries



import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import tensorflow as tf
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt


### Dataset

In [2]:
#load data

df = pd.read_csv("AAPL.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,79.117142,79.285713,77.375717,78.432854,69.319641,140129500
1,2013-01-03,78.268570,78.524284,77.285713,77.442856,68.444672,88241300
2,2013-01-04,76.709999,76.947144,75.118568,75.285713,66.538162,148583400
3,2013-01-07,74.571426,75.614288,73.599998,74.842857,66.146751,121039100
4,2013-01-08,75.601425,75.984283,74.464287,75.044289,66.324783,114676800


In [4]:
#extract only numbers in OPEN

apple_training_processed = df.iloc[:, 1:2].values

In [5]:
apple_training_processed

array([[ 79.117142],
       [ 78.26857 ],
       [ 76.709999],
       ...,
       [170.100006],
       [171.      ],
       [170.520004]])

### Normalisation 

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

apple_training_scaled = scaler.fit_transform(apple_training_processed)

In [7]:
apple_training_scaled

array([[0.19795893],
       [0.19086893],
       [0.17784673],
       ...,
       [0.95814041],
       [0.96566005],
       [0.96164958]])

In [8]:
y = len(apple_training_scaled)
y

1259

### Convert Training Data to Right Shape
As I said earlier, in a time series problems, we have to predict a value at time T, based on the data from days T-N where N can be any number of steps. In this article, we are going to predict the opening stock price of the data based on the opening stock prices for the past 60 days. I have tried and tested different numbers and found that the best results are obtained when past 60 time steps are used. You can try different numbers and see how your algorithm performs.

Our feature set should contain the opening stock price values for the past 60 days while the label or dependent variable should be the stock price at the 61st day. Execute the following script to create feature and label set.



In [9]:
features_set = []
labels = []
for i in range (60,1259):
    features_set.append(apple_training_scaled[i-60:i,0])
    labels.append(apple_training_scaled[i,0])

In [10]:
len(features_set)

1199

In [11]:
len(labels)

1199

We need to convert both the feature_set and the labels list to the numpy array before we can use it for training. Execute the following script:

In [12]:
features_set, labels = np.array(features_set), np.array(labels)

In order to train LSTM on our data, we need to convert our data into the shape accepted by the LSTM. We need to convert our data into three-dimensional format. The first dimension is the number of records or rows in the dataset which is 1260 in our case. The second dimension is the number of time steps which is 60 while the last dimension is the number of indicators. Since we are only using one feature, i.e Open, the number of indicators will be one. Execute the following script:

In [13]:
features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))

### Training The LSTM
We have preprocessed our data and have converted it into the desired format. Now is the time to create our LSTM. The LSTM model that we are going to create will be a sequential model with multiple layers. We will add four LSTM layers to our model followed by a dense layer that predicts the future stock price.



In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


As a first step, we need to instantiate the Sequential class. This will be our model class and we will add LSTM, Dropout and Dense layers to this model. Execute the following script

In [15]:
model = Sequential()

### Creating LSTM and Dropout Layers
Let's add LSTM layer to the model that we just created.

To add a layer to the sequential model, the add method is used. Inside the add method, we passed our LSTM layer. The first parameter to the LSTM layer is the number of neurons or nodes that we want in the layer. The second parameter is return_sequences, which is set to true since we will add more layers to the model. The first parameter to the input_shape is the number of time steps while the last parameter is the number of indicators.

In [16]:
model.add(LSTM(units=10, return_sequences=True, input_shape=(features_set.shape[1], 1)))

Instructions for updating:
Colocations handled automatically by placer.


#### Add dropout layer to avoid overfitting

Let's now add a dropout layer to our model. Dropout layer is added to avoid over-fitting, which is a phenomenon where a machine learning model performs better on the training data compared to the test data. Execute the following script to add dropout layer.



In [17]:
model.add(Dropout(0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### Add more layers

Let's add three more LSTM and dropout layers to our model. Run the following script.

In [18]:
model.add(LSTM(units=10, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=10, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=10))
model.add(Dropout(0.2))

### Creating Dense Layer
To make our model more robust, we add a dense layer at the end of the model. The number of neurons in the dense layer will be set to 1 since we want to predict a single value in the output.



In [19]:
model.add(Dense(units = 1))

### Model Compilation
Finally, we need to compile our LSTM before we can train it on the training data. The following script compiles the our model.

We call the compile method on the Sequential model object which is "model" in our case. We use the mean squared error as loss function and to reduce the loss or to optimize the algorithm, we use the adam optimizer.

In [20]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

### Algorithm Training
Now is the time to train the model that we defined in the previous few steps. To do so, we call the fit method on the model and pass it our training features and labels as shown below:

In [ ]:
model.fit(features_set, labels, epochs = 100, batch_size = 32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100


In [ ]:
print('test')